# Téléchargement des données de stations météorologiques d'Environnement Canada

### Préparé par : Sébastien Langlois

In [1]:
import pandas as pd
import datetime
import matplotlib.pyplot as plt
from dateutil import rrule
from datetime import datetime, timedelta
from bs4 import BeautifulSoup
import requests
import re
import os
from dask.distributed import Client
from retrying import retry
import requests
from io import StringIO
import multiprocessing
from dask import dataframe as dd 
import glob
from dask.diagnostics import ProgressBar
import sys

# Local functions
sys.path.append(os.path.abspath(os.path.join('..','..')))
import sefm.database.datascraper.ec as ec

# Client Dask

In [2]:
client = Client(n_workers=int(multiprocessing.cpu_count()*2))
client

Client Scheduler: tcp://127.0.0.1:42927 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 64 Cores: 64 Memory: 135.11 GB


# Liste des stations météo - Environnement Canada

In [3]:
dwn_url='https://drive.google.com/uc?export=download&id=' + '1egfzGgzUb0RFu_EE5AYFZtsyXPfZ11y2'
df = pd.read_csv(StringIO(requests.get(dwn_url).text), skiprows=3)
df_stations = df[df.Province.isin(['QUEBEC','ONTARIO','NEW BRUNSWICK', 'NOVA SCOTIA','NEWFOUNDLAND'])]
ddf = dd.from_pandas(df_stations, npartitions=16)
ddf

,Name,Province,Climate ID,Station ID,WMO ID,TC ID,Latitude (Decimal Degrees),Longitude (Decimal Degrees),Latitude,Longitude,Elevation (m),First Year,Last Year,HLY First Year,HLY Last Year,DLY First Year,DLY Last Year,MLY First Year,MLY Last Year
npartitions=16,,,,,,,,,,,,,,,,,,,
5102,object,object,object,int64,float64,object,float64,float64,int64,int64,float64,int64,int64,float64,float64,float64,float64,float64,float64
5328,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8548,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8764,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


# Extractions des séries temporelles (.csv)

In [4]:
output_path_hourly = '/media/slanglois/ssd-2tb/EC/hourly'
output_path_daily = '/media/slanglois/ssd-2tb/EC/daily'

### Hourly data

In [ ]:
ddf.apply(ec.getCompleteWeatherTimeSeriesEC, axis=1,
          output_path=output_path_hourly, timeframe=1).compute()

### Daily data

In [5]:
ddf.apply(ec.getCompleteWeatherTimeSeriesEC, axis=1,
          output_path=output_path_daily, timeframe=2, meta=('int')).compute()

5102    None
5103    None
5104    None
5105    None
5106    None
        ... 
8760    None
8761    None
8762    None
8763    None
8764    None
Length: 3607, dtype: object